<a href="https://colab.research.google.com/github/kaledai069/Crossword-Generator/blob/master/Grid_answer_clue_pair_generator_Approach_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!git clone https://github.com/kaledai069/Crossword-Solver-A-Neural-Transformer-based-Approach
%cd Crossword-Solver-A-Neural-Transformer-based-Approach/
!bash inference_setup.sh

Cloning into 'Crossword-Solver-A-Neural-Transformer-based-Approach'...
remote: Enumerating objects: 1919, done.
remote: Counting objects: 100% (619/619), done.
remote: Compressing objects: 100% (287/287), done.
remote: Total 1919 (delta 389), reused 554 (delta 328), pack-reused 1300
Receiving objects: 100% (1919/1919), 13.39 MiB | 20.32 MiB/s, done.
Resolving deltas: 100% (583/583), done.
/content/Crossword-Solver-A-Neural-Transformer-based-Approach
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 34.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 14.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 35.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━

In [ ]:
%cd Crossword-Solver-A-Neural-Transformer-based-Approach/

/content/Crossword-Solver-A-Neural-Transformer-based-Approach


In [ ]:
import json
import os
import pandas as pd
import random
from pprint import pprint


In [ ]:
CA_DATASET = "/content/gdrive/MyDrive/Clue-Answer-Dataset_Version_2.csv"

clue_answer_df = pd.read_csv(CA_DATASET)
clue_answer_df['Unsegmented Answer'] = clue_answer_df['Segmented Answer'].apply(lambda x: x.replace(' ', '') if not isinstance(x, float) else x)

all_clues = clue_answer_df['Clue'].tolist()
all_answers = clue_answer_df['Unsegmented Answer'].tolist()
clue_answer_df.head()

,Unnamed: 0,Clue,Answer,Segmented Answer,answer_length,Unsegmented Answer
0,0,cpa for example crossword clue,taxpro,tax pro,6.0,taxpro
1,1,cordial crossword clue,warm,warm,4.0,warm
2,2,joe without jolt crossword clue,decaf,decaf,5.0,decaf
3,3,bouillabaisse bit crossword clue,mussel,mussel,6.0,mussel
4,4,pre-gps travel aid crossword clue,aaamap,aaa map,6.0,aaamap


In [ ]:
def get_me_a_random_clue(answer_length):
    global all_clues

    random_index = random.randint(0, len(all_clues))
    while 'across' in all_clues[random_index] or 'down' in all_clues[random_index]:
        random_index = random.randint(0, len(all_clues))
    return all_clues[random_index]

In [ ]:
def find_grid_position(grid_nums, clue_num, n_rows):
    reshaped_grid = [grid_nums[i:i + n_rows] for i in range(0, len(grid_nums), n_rows)]

    for i in range(len(reshaped_grid)):
        for j in range(len(reshaped_grid[0])):
            if reshaped_grid[i][j] == clue_num:
                return (i, j)

def find_answer_length(main_grid, position, grid_size, across = True):
    rows, cols = grid_size
    i, j = position
    count = 0
    if across:
        for idx in range(j, cols):
            if main_grid[i][idx] == '.':
                return count
            else:
                count += 1
    else:
        for idx in range(i, rows):
            if main_grid[idx][j] == '.':
                return count
            else:
                count += 1
    return count

def json_CA_json_converter(json_file_path, is_path):
    if is_path:
        with open(json_file_path, "r") as file:
            data = json.load(file)
    else:
        data = json_file_path

    json_conversion_dict = {}

    rows = data["size"]["rows"]
    cols = data["size"]["cols"]

    grid_list = data['grid']

    main_grid = [grid_list[i:i + rows] for i in range(0, len(grid_list), rows)]


    json_conversion_dict["metadata"] = {"rows": rows, "cols": cols}

    across_grid_nums = data['across_nums']
    down_grid_nums = data['down_nums']

    across_clue_answer = {}
    down_clue_answer = {}

    grid_info = data["grid"]
    grid_num = data["gridnums"]
    reshaped_grid = [grid_num[i:i + rows] for i in range(0, len(grid_num), rows)]

    for clue_num in across_grid_nums:
      position = find_grid_position(grid_num, clue_num, rows)
      answer_count = find_answer_length(main_grid, position, (rows, cols))
      random_clue = get_me_a_random_clue(answer_count)
      across_clue_answer[str(clue_num)] = [random_clue, "".join(['A' for _ in range(answer_count)])]

    for clue_num in down_grid_nums:
      position = find_grid_position(grid_num, clue_num, rows)
      answer_count = find_answer_length(main_grid, position, (rows, cols), False)
      random_clue = get_me_a_random_clue(answer_count)
      down_clue_answer[str(clue_num)] = [random_clue, "".join(['A' for _ in range(answer_count)])]

    json_conversion_dict["clues"] = {
          "across": across_clue_answer,
          "down": down_clue_answer,
      }

    grid_info_list = []
    for i in range(rows):
        row_list = []
        for j in range(cols):
            if grid_info[i * rows + j] == ".":
                row_list.append("BLACK")
            else:
                if grid_num[i * rows + j] == 0:
                    row_list.append(["", grid_info[i * rows + j]])
                else:
                    row_list.append(
                        [str(grid_num[i * rows + j]), grid_info[i * rows + j]]
                    )
        grid_info_list.append(row_list)

    json_conversion_dict["grid"] = grid_info_list

    return json_conversion_dict

## Inference with the crossword solver

#### Important imports for solver

In [ ]:
import datetime
import time
import matplotlib.pyplot as plt
import cv2
import pprint
import shutil
import os

from copy import deepcopy
from pprint import pprint
from matplotlib.gridspec import GridSpec

from solver.Crossword import Crossword
from solver.BPSolver import BPSolver
from utils import puz_to_json, fetch_nyt_crossword
from draw_grid import get_grid


In [ ]:
model_path = "/content/gdrive/MyDrive/First Pass Model/distilbert_EPOCHs_7_COMPLETE.bin"
ans_path = "/content/gdrive/MyDrive/First Pass Model/all_answer_list.tsv"
dense_embd_path = "/content/gdrive/MyDrive/First Pass Model/distilbert_7_epochs_embeddings.pkl"

reranker_path = "/content/gdrive/MyDrive/Second Pass Model/t5_word_segmented_pair_2_epochs"
reranker_model_type = 't5-small'

In [ ]:
def solve(crossword, m_path, a_path, d_path, r_path, reranker_m_type, max_candidate = 50000, score_threshold = 0.5):
    solver = BPSolver(
                      crossword,
                      model_path = m_path,
                      ans_tsv_path = a_path,
                      dense_embd_path = d_path,
                      reranker_path = r_path,
                      reranker_model_type = reranker_m_type,
                      max_candidates = max_candidate,
                      score_improvement_threshold = score_threshold
                      )

    solution = solver.solve(num_iters = 60, iterative_improvement_steps = 0)
    return solution

In [ ]:
json_puzzle_path = "/content/crossword_json_15x15_2.json"
puzzle = json_CA_json_converter(json_puzzle_path, True)

crossword = Crossword(puzzle)
start_t = time.time()
output = solve(crossword, model_path, ans_path, dense_embd_path, reranker_path, "t5-small", 50000, 0.5)
end_t = time.time()

print("Total time taken: ", end_t - start_t)


Beginning Belief Propagation iteration steps


100%|███████████████████████████████████████████████████████████████| 60/60 [00:20<00:00,  2.92it/s]


Belief Propagation iteration complete


Before II with T5-small: Letters Correct: 0.0% | Words Correct: 0.0%
Total time taken:  79.27632594108582


In [ ]:
first_pass_grid = output['first pass model']['grid']
# second_pass_grid = output['second pass model']['all grids'][0]

pprint(first_pass_grid)

clue_answer_pair = {}
generated_answer_list = []

test_grid = first_pass_grid
for key, var in crossword.variables.items():
  cells_val = var['cells']
  word = ''
  for cell in cells_val:
    word += first_pass_grid[cell[0]][cell[1]]
  generated_answer_list.append(word)
  clue_answer_pair[key] = {
      'clue': var['clue'],
      'answer': word
  }

print("Total answers: ", len(generated_answer_list))

found_answer = []
for answer in generated_answer_list:
    if answer.lower() in all_answers:
        print(answer)
        # print(clue_answer_df[clue_answer_df['Unsegmented Answer'] == answer.lower()]['Clue'].tolist())
        found_answer.append(answer)

print("Actual answers found: ", len(found_answer))

[['N', 'S', 'E', 'K', 'T', '', 'T', 'A', 'S', '', '', 'T', 'D', 'D', 'S'],
 ['E', 'L', 'C', 'E', 'E', '', 'T', 'M', 'E', 'S', 'B', 'E', 'A', 'E', 'E'],
 ['R', 'O', 'A', 'N', 'N', '', 'T', 'M', 'E', 'C', 'I', 'C', 'I', 'N', 'S'],
 ['O', 'O', 'S', 'T', '', 'G', 'E', 'O', 'E', 'A', 'D', '', 'S', 'E', 'E'],
 ['', 'P', 'H', 'E', 'N', 'O', 'M', '', 'Y', 'T', 'E', '', 'M', 'U', 'I'],
 ['', '', '', '', 'C', 'R', 'E', 'L', 'E', 'T', '', 'G', 'I', 'R', 'L'],
 ['', '', 'O', 'W', 'E', 'D', 'T', 'O', '', 'S', 'L', 'U', 'D', 'G', 'E'],
 ['S', 'E', 'T', 'I', 'N', '', 'L', 'A', 'O', '', 'E', 'R', 'E', 'P', 'T'],
 ['C', 'R', 'O', 'R', 'E', 'S', '', 'D', 'U', 'S', 'T', 'B', 'L', '', ''],
 ['R', 'I', 'N', 'K', '', 'O', 'B', 'S', 'T', 'E', 'T', '', '', '', ''],
 ['A', 'L', 'Y', '', 'B', 'E', 'T', '', 'T', 'G', 'E', 'C', 'A', 'T', ''],
 ['T', 'R', 'F', '', 'G', 'L', 'E', 'N', 'D', 'A', '', 'R', 'L', 'T', 'S'],
 ['C', 'O', 'I', 'N', 'T', 'R', 'E', 'A', 'U', '', 'A', 'O', 'D', 'T', 'Y'],
 ['T', 'H', 'E', 'M'